In [ ]:
from torchvision import datasets
train_dataset = datasets.CIFAR10(root='./CIFAR10', train=True, download=True)
test_dataset = datasets.CIFAR10(root='./CIFAR10', train=False, download=True)
print(f"# of images (train): {len(train_dataset)}")
print(f"# of images (test): {len(test_dataset)}")
print(f"# of classes: {len(train_dataset.classes)}")
print(f"size of 1st image: {train_dataset[0][0].size}")

In [ ]:
import matplotlib.pyplot as plt
image, label = train_dataset[0]
plt.imshow(image)
plt.title(f'Label: {label} ({train_dataset.classes[label]})')
plt.axis('off')
plt.show()

In [ ]:
classes = train_dataset.classes
fig, axis = plt.subplots(2, 5, figsize=(10, 5))
for i in range(10):
    for image, label in train_dataset:
        if label == i:
            ax = axis[i // 5, i % 5]
            ax.set_title(f"Label: {int(label)}")
            ax.axis('off')
            break
plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.linear1 = nn.Linear(28*28, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

        self.classifier = nn.Linear(in_features=4*4*128, out_features=num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
model = SimpleMLP()

In [ ]:
from torchvision.transforms.functional import to_tensor
image, label = train_dataset[0]
sample = to_tensor(image).unsqueeze(0)
sample = (sample - 0.5) / 0.5

In [ ]:
logits = model(sample)
probs = F.softmax(logits, dim=1)

In [ ]:
probs_np = probs.detach().numpy().flatten()
plt.bar(range(10), probs_np)
plt.ylabel('Probability')
plt.xticks(range(10))
plt.xlabel('Classes')
plt.ylim(0, 1)
plt.show()

In [ ]:
probs_np = probs.detach().numpy().flatten()
plt.bar(range(10), probs_np)
plt.ylabel('Probability')
plt.xticks(range(10), classes) # 変更
plt.xticks(rotation=45) # 変更
plt.xlabel('Classes')
plt.ylim(0, 1)
plt.show()

In [ ]:
predicted_class = torch.argmax(probs, dim=1).item()
predicted_name = classes[predicted_class]
print(f"Predicted class: {predicted_class} ({predicted_name})")